<a href="https://colab.research.google.com/github/kiyaka-990/splitmate/blob/main/Another_copy_of_level1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import qrcode
from PIL import Image
import io
from google.colab import files
import os
from qrcode.image.svg import SvgImage
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas

def create_qrcode():
    """
    Creates a customizable QR code based on user input and allows download.
    """
    print("Welcome to the QR Code Generator!")
    print("Choose the type of content for your QR code:")
    print("1. Plain Text")
    print("2. Website URL (Multiple links not directly supported by a single QR, use a landing page)")
    print("3. Phone Contact (vCard format)")
    print("4. App (Redirect to app store - Requires a redirect URL)")
    print("5. SMS (Link phone number for texting)")
    print("6. Email (Link email address)")
    print("7. Phone Call (Link phone number)")
    print("8. Downloadable PDF (Requires a direct link to the PDF)")
    print("9. Exit") # Added exit option

    while True:
        choice = input("Enter the number of your choice: ")
        if choice in ['1', '2', '3', '4', '5', '6', '7', '8']:
            break
        elif choice == '9': # Handle exit option
            print("Exiting QR Code Generator.")
            return # Exit the function
        else:
            print("Invalid choice. Please enter a number from 1 to 9.") # Updated prompt

    data = ""
    if choice == '1':
        data = input("Enter the plain text: ")
    elif choice == '2':
        data = input("Enter the website URL: ")
    elif choice == '3':
        name = input("Enter contact name: ")
        phone = input("Enter phone number: ")
        email = input("Enter email address (optional): ")
        data = f"BEGIN:VCARD\nVERSION:3.0\nN:{name}\nTEL:{phone}"
        if email:
            data += f"\nEMAIL:{email}"
        data += "\nEND:VCARD"
    elif choice == '4':
        data = input("Enter the app store redirect URL: ")
    elif choice == '5':
        phone_number = input("Enter the phone number for SMS: ")
        message = input("Enter the pre-filled message (optional): ")
        data = f"sms:{phone_number}"
        if message:
            data += f"?body={message}"
    elif choice == '6':
        email_address = input("Enter the email address: ")
        subject = input("Enter the email subject (optional): ")
        body = input("Enter the email body (optional): ")
        data = f"mailto:{email_address}"
        params = []
        if subject:
            params.append(f"subject={subject}")
        if body:
            params.append(f"body={body}")
        if params:
            data += "?" + "&".join(params)
    elif choice == '7':
        phone_number = input("Enter the phone number for calls: ")
        data = f"tel:{phone_number}"
    elif choice == '8':
        data = input("Enter the direct URL to the PDF: ")

    # Create QR code instance
    qr = qrcode.QRCode(
        version=1,
        error_correction=qrcode.constants.ERROR_CORRECT_H,
        box_size=10,
        border=4,
    )
    qr.add_data(data)
    qr.make(fit=True)

    # Get colors from user with validation
    while True:
        fill_color = input("Enter the fill color (e.g., black, #000000): ")
        if fill_color:
            break
        else:
            print("Fill color cannot be empty. Please enter a color.")

    while True:
        back_color = input("Enter the background color (e.g., white, #FFFFFF): ")
        if back_color:
            break
        else:
            print("Background color cannot be empty. Please enter a color.")


    img = qr.make_image(fill_color=fill_color, back_color=back_color).convert('RGB')

    # Customize QR code (logo)
    add_logo = input("Do you want to add a logo? (yes/no): ").lower()
    if add_logo == 'yes':
        logo_path = input("Enter the path to your logo image file (e.g., logo.png): ")

        if logo_path:
            try:
                logo = Image.open(logo_path)
                # Resize logo
                logo_size = int(img.size[0] * 0.3) # Adjust logo size as a percentage of QR code size
                logo = logo.resize((logo_size, logo_size), Image.Resampling.LANCZOS)

                # Calculate position for logo (center)
                pos = ((img.size[0] - logo.size[0]) // 2, (img.size[1] - logo.size[1]) // 2)

                img.paste(logo, pos)
                print("Logo added successfully.")
            except FileNotFoundError:
                print(f"Error: Logo file not found at {logo_path}")
            except Exception as e:
                print(f"Could not add logo. Error: {e}")


    # Select download format
    print("\nSelect the download format:")
    print("1. PNG (Portable Network Graphics)")
    print("2. SVG (Scalable Vector Graphics)")
    print("3. PDF (Portable Document Format)")

    while True:
        format_choice = input("Enter the number of your choice: ")
        if format_choice in ['1', '2', '3']:
            break
        else:
            print("Invalid choice. Please enter a number from 1 to 3.")

    filename = "qrcode"
    if format_choice == '1':
        filename += ".png"
        img.save(filename, format='PNG') # Save the image to a file
        files.download(filename) # Download the saved file
        print(f"PNG file '{filename}' downloaded.")
    elif format_choice == '2':
        # SVG requires a different approach, generating SVG XML
        try:
            from qrcode.image.svg import SvgImage
            qr_svg = qrcode.QRCode(
                version=1,
                error_correction=qrcode.constants.ERROR_CORRECT_H,
                box_size=10,
                border=4,
                image_factory=SvgImage
            )
            qr_svg.add_data(data)
            qr_svg.make(fit=True)
            svg_img = qr_svg.make_image(fill_color=fill_color, back_color=back_color) # Apply background color to SVG
            filename += ".svg"
            svg_img.save(filename)
            files.download(filename)
            print(f"SVG file '{filename}' downloaded.")
        except ImportError:
            print("SVG generation requires the 'svgwrite' library. Install with: !pip install svgwrite")
        except Exception as e:
            print(f"Could not generate SVG. Error: {e}")

    elif format_choice == '3':
        # PDF generation
        try:
            from reportlab.lib.pagesizes import letter
            from reportlab.pdfgen import canvas

            filename += ".pdf"
            c = canvas.Canvas(filename, pagesize=letter)

            # Save PIL image to a temporary file to be used by ReportLab
            temp_img_path = "temp_qr.png"
            img.save(temp_img_path, format='PNG')

            # Calculate image position to center it on the page
            img_width, img_height = img.size
            page_width, page_height = letter
            x_pos = (page_width - img_width) / 2
            y_pos = (page_height - img_height) / 2

            c.drawImage(temp_img_path, x_pos, y_pos, width=img_width, height=img_height)
            c.save()

            # Clean up temporary file
            import os
            os.remove(temp_img_path)

            files.download(filename)
            print(f"PDF file '{filename}' downloaded.")
        except ImportError:
            print("PDF generation requires the 'reportlab' library. Install with: !pip install reportlab")
            print("Please install it and run the code again to generate the PDF.")
        except Exception as e:
            print(f"Could not generate PDF. Error: {e}")


# Run the function
create_qrcode()

Welcome to the QR Code Generator!
Choose the type of content for your QR code:
1. Plain Text
2. Website URL (Multiple links not directly supported by a single QR, use a landing page)
3. Phone Contact (vCard format)
4. App (Redirect to app store - Requires a redirect URL)
5. SMS (Link phone number for texting)
6. Email (Link email address)
7. Phone Call (Link phone number)
8. Downloadable PDF (Requires a direct link to the PDF)
9. Exit
Enter the number of your choice: 9
Exiting QR Code Generator.
